### 长短期记忆（LSTM）


本节将介绍另一种常用的门控循环神经网络：长短期记忆（long short-term memory，LSTM）[1]。它比门控循环单元的结构稍微复杂一点。


#### 长短期记忆

LSTM 中引入了3个门，即输入门（input gate）、遗忘门（forget gate）和输出门（output gate），以及与隐藏状态形状相同的记忆细胞（某些文献把记忆细胞当成一种特殊的隐藏状态），从而记录额外的信息。


##### 输入门、遗忘门和输出门

与门控循环单元中的重置门和更新门一样，如图6.7所示，长短期记忆的门的输入均为当前时间步输入$\boldsymbol{X}_t$与上一时间步隐藏状态$\boldsymbol{H}_{t-1}$，输出由激活函数为sigmoid函数的全连接层计算得到。如此一来，这3个门元素的值域均为$[0,1]$。

![长短期记忆中输入门、遗忘门和输出门的计算](./img/lstm_0.svg)

具体来说，假设隐藏单元个数为$h$，给定时间步$t$的小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$（样本数为$n$，输入个数为$d$）和上一时间步隐藏状态$\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$。
时间步$t$的输入门$\boldsymbol{I}_t \in \mathbb{R}^{n \times h}$、遗忘门$\boldsymbol{F}_t \in \mathbb{R}^{n \times h}$和输出门$\boldsymbol{O}_t \in \mathbb{R}^{n \times h}$分别计算如下：

$$
\begin{aligned}
\boldsymbol{I}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xi} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hi} + \boldsymbol{b}_i),\\
\boldsymbol{F}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xf} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hf} + \boldsymbol{b}_f),\\
\boldsymbol{O}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xo} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{ho} + \boldsymbol{b}_o),
\end{aligned}
$$

其中的$\boldsymbol{W}_{xi}, \boldsymbol{W}_{xf}, \boldsymbol{W}_{xo} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hi}, \boldsymbol{W}_{hf}, \boldsymbol{W}_{ho} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_i, \boldsymbol{b}_f, \boldsymbol{b}_o \in \mathbb{R}^{1 \times h}$是偏差参数。


### 候选记忆细胞

接下来，长短期记忆需要计算候选记忆细胞$\tilde{\boldsymbol{C}}_t$。它的计算与上面介绍的3个门类似，但使用了值域在$[-1, 1]$的tanh函数作为激活函数，如图6.8所示。

![长短期记忆中候选记忆细胞的计算](./img/lstm_1.svg)


具体来说，时间步$t$的候选记忆细胞$\tilde{\boldsymbol{C}}_t \in \mathbb{R}^{n \times h}$的计算为

$$\tilde{\boldsymbol{C}}_t = \text{tanh}(\boldsymbol{X}_t \boldsymbol{W}_{xc} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hc} + \boldsymbol{b}_c),$$

其中$\boldsymbol{W}_{xc} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hc} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_c \in \mathbb{R}^{1 \times h}$是偏差参数。


#### 记忆细胞

我们可以通过元素值域在$[0, 1]$的输入门、遗忘门和输出门来控制隐藏状态中信息的流动，这一般也是通过使用按元素乘法（符号为$\odot$）来实现的。当前时间步记忆细胞$\boldsymbol{C}_t \in \mathbb{R}^{n \times h}$的计算组合了上一时间步记忆细胞和当前时间步候选记忆细胞的信息，并通过遗忘门和输入门来控制信息的流动：

$$\boldsymbol{C}_t = \boldsymbol{F}_t \odot \boldsymbol{C}_{t-1} + \boldsymbol{I}_t \odot \tilde{\boldsymbol{C}}_t.$$


如图6.9所示，遗忘门控制上一时间步的记忆细胞$\boldsymbol{C}_{t-1}$中的信息是否传递到当前时间步，而输入门则控制当前时间步的输入$\boldsymbol{X}_t$通过候选记忆细胞$\tilde{\boldsymbol{C}}_t$如何流入当前时间步的记忆细胞。如果遗忘门一直近似1且输入门一直近似0，过去的记忆细胞将一直通过时间保存并传递至当前时间步。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

![长短期记忆中记忆细胞的计算。这里的$\odot$是按元素乘法](./img/lstm_2.svg)


#### 隐藏状态

有了记忆细胞以后，接下来我们还可以通过输出门来控制从记忆细胞到隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$的信息的流动：

$$\boldsymbol{H}_t = \boldsymbol{O}_t \odot \text{tanh}(\boldsymbol{C}_t).$$

这里的tanh函数确保隐藏状态元素值在-1到1之间。需要注意的是，当输出门近似1时，记忆细胞信息将传递到隐藏状态供输出层使用；当输出门近似0时，记忆细胞信息只自己保留。图6.10展示了长短期记忆中隐藏状态的计算。

![长短期记忆中隐藏状态的计算。这里的$\odot$是按元素乘法](./img/lstm_3.svg)

### 怎么理解LSTM？为什么它可以解决梯度弥散或者梯度爆炸问题？

当然，LSTM和基线RNN并没有特别大的结构不同，但是它们用了不同的函数来计算隐状态。LSTM的“记忆”我们叫做细胞/cells，你可以直接把它们想做黑盒，这个黑盒的输入为前状态和当前输入。这些“细胞”会决定哪些之前的信息和状态需要保留/记住，而哪些要被抹去。实际的应用中发现，这种方式可以有效地保存很长时间之前的关联信息。

关键就在于它的几个门，可以控制是否记住原来的状态信息。

比如：你浏览评论时，你的大脑下意识地只会记住重要的关键词，比如“amazing”和“awsome”这样的词汇，而不太会关心“this”、“give”、“all”、“should”等字样。如果朋友第二天问你用户评价都说了什么，那你可能不会一字不漏地记住它，而是会说出但大脑里记得的主要观点，比如“下次肯定还会来买”，那其他一些无关紧要的内容自然会从记忆中逐渐消失。

<img src="img/quesbase64155693990149080320.png">

    上图中，σ表示的Sigmoid 激活函数与 tanh 函数类似，不同之处在于 sigmoid 是把值压缩到0~1 之间而不是 -1~1 之间。这样的设置有助于更新或忘记信息：

    因为任何数乘以 0 都得 0，这部分信息就会剔除掉；
    同样的，任何数乘以 1 都得到它本身，这部分信息就会完美地保存下来。



#### 细胞状态
LSTM的关键就是细胞状态C，水平线在图上方贯穿运行。

LSTM有通过精心设计的称作为“门”的结构来去除或者增加信息到细胞状态的能力。门是一种让信息选择式通过的方法。他们包含一个sigmoid神经网络层和一个pointwise乘法的非线性操作。

c_t 综合了当前词 x_t 和前一时刻细胞状态 c_(t-1) 的信息。这和 ResNet 中的残差逼近思想十分相似，通过从 c_(t-1) 到 c_t 的「短路连接」，梯度得已有效地反向传播。当 f_t 处于闭合状态时，c_t 的梯度可以直接沿着最下面这条短路线传递到c_(t-1)，不受参数 W_(xh) 和 W_(hh) 的影响，这是 LSTM 能有效地缓解梯度消失现象的关键所在。


#### 忘记门

我们LSTM中的第一步是决定我们会从细胞状态中丢弃什么信息。
f_t 控制上一时刻细胞状态 c_(t-1) 的信息融入细胞状态 c_t。在理解一句话时，当前词 x_t 可能继续延续上文的意思继续描述，也可能从当前词 x_t 开始描述新的内容，与上文无关。和输入门 i_t 相反，f_t 不对当前词 x_t 的重要性作判断，而判断的是上一时刻的细胞状态c_(t-1)对计算当前细胞状态 c_t 的重要性。当 f_t 开关打开的时候，网络将不考虑上一时刻的细胞状态 c_(t-1)。
<img src="img/quesbase64155698137271488143.gif">

#### 输入门
    下一步是确定什么样的新信息被存放在细胞状态中。这里包含两个部分：
    第一，sigmoid层称“输入门层”决定什么值我们将要更新；
    第二，一个tanh层创建一个新的候选值向量，会被加入到状态中。
    下一步，我们会讲这两个信息来产生对状态的更新。

    i_t 控制当前词 x_t 的信息融入细胞状态 c_t。在理解一句话时，当前词 x_t 可能对整句话的意思很重要，也可能并不重要。输入门的目的就是判断当前词 x_t 对全局的重要性。当 i_t 开关打开的时候，网络将不考虑当前输入  x_t。


###### 更新细胞状态
现在是更新旧细胞状态的时间了，$C_{t-1}$更新为$C_t$。前面的步骤已经决定了将会做什么，我们现在就是实际去完成。
我们把旧状态与$f_t$相乘，丢弃掉我们确定需要丢弃的信息。接着加上$i_t * C_t$。这就是新的候选值，根据我们决定更新每个状态的程度进行变化。
在语言模型的例子中，这就是我们实际根据前面确定的目标，丢弃旧代词的性别信息并添加新的信息的地方，类似更新细胞状态。

#### 输出门
最终，我们需要确定输出什么值。这个输出将会基于我们的细胞状态，但是也是一个过滤后的版本。

    首先，我们运行一个sigmoid层来确定细胞状态的哪个部分将输出出去。
    接着，我们把细胞状态通过tanh进行处理（得到一个在-1到1之间的值）并将它和sigmoid门的输出相乘，最终我们仅仅会输出我们确定输出的那部分。

    在语言模型的例子中，因为他就看到了一个代词，可能需要输出与一个动词相关的信息。例如，可能输出是否代词是单数还是负数，这样如果是动词的话，我们也知道动词需要进行的词形变化，进而输出信息。

    输出门的目的是从细胞状态 c_(t-1) 产生隐层单元 h_(t-1)。并不是 c_(t-1) 中的全部信息都和隐层单元 h_(t-1) 有关，c_(t-1) 可能包含了很多对 h_(t-1) 无用的信息。因此，o_t 的作用就是判断 c_(t-1) 中哪些部分是对 h_(t-1) 有用的，哪些部分是无用的。
    
 <img src="img/quesbase6415569403763141128.png">

为了解决标准 RNN 的梯度消失问题，GRU 使用了更新门（update gate）与重置门（reset gate）。基本上，这两个门控向量决定了哪些信息最终能作为门控循环单元的输出。这两个门控机制的特殊之处在于，它们能够保存长期序列中的信息，且不会随时间而清除或因为与预测不相关而移除。

In [4]:
import numpy as np

vocab_size = 10000
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        return np.random.normal(scale=0.01, shape=shape)

    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                np.zeros(num_hiddens))

    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = np.zeros(num_outputs)
    # 附上梯度
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc,
              b_c, W_hq, b_q]
   
    return params

In [5]:
def init_lstm_state(batch_size, num_hiddens):
    return (np.zeros(shape=(batch_size, num_hiddens)),
            np.zeros(shape=(batch_size, num_hiddens)))

In [6]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c,
     W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = np.sigmoid(nd.dot(X, W_xi) + np.dot(H, W_hi) + b_i)
        F = np.sigmoid(nd.dot(X, W_xf) + np.dot(H, W_hf) + b_f)
        O = np.sigmoid(nd.dot(X, W_xo) + np.dot(H, W_ho) + b_o)
        C_tilda = np.tanh(nd.dot(X, W_xc) + np.dot(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = np.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [13]:
import  keras

keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', 
                  use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                  bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, 
                  bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, 
                  bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=False, 
                  return_state=False, go_backwards=False, stateful=False, unroll=False)

#### 小结

* 长短期记忆的隐藏层输出包括隐藏状态和记忆细胞。只有隐藏状态会传递到输出层。
* 长短期记忆的输入门、遗忘门和输出门可以控制信息的流动。
* 长短期记忆可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。



#### 练习

* 调节超参数，观察并分析对运行时间、困惑度以及创作歌词的结果造成的影响。
* 在相同条件下，比较长短期记忆、门控循环单元和不带门控的循环神经网络的运行时间。
* 既然候选记忆细胞已通过使用tanh函数确保值域在-1到1之间，为什么隐藏状态还需要再次使用tanh函数来确保输出值域在-1到1之间？